In [3]:
!python3 track_football.py --source /home/andi/FER/year5/PlayerTracking/sequences/demo_cut.mp4 \
--referee-color red --team1-color white --team2-color black --max-frames 50 \
--classes 0 32 --save-vid --show-vid --save-txt --save-crop

File: /home/andi/FER/year5/PlayerTracking/tracker/yolov5/utils/general.py
ROOT: /home/andi/FER/year5/PlayerTracking/tracker/yolov5
File: /home/andi/FER/year5/PlayerTracking/tracker/yolov5/utils/general.py
ROOT: /home/andi/FER/year5/PlayerTracking/tracker/yolov5
track_football: yolo_weights=/home/andi/FER/year5/PlayerTracking/tracker/weights/yolov5m.pt, appearance_descriptor_weights=/home/andi/FER/year5/PlayerTracking/tracker/weights/osnet_x0_25_msmt17.pt, tracking_method=strongsort, source=/home/andi/FER/year5/PlayerTracking/sequences/demo_cut.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=True, save_txt=True, save_conf=False, save_crop=True, save_vid=True, nosave=False, classes=[0, 32], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False, eval=False, max_frames=50, referee_color=red, team1_color

In [ ]:
# Just testing nothing too smart to see here

import cv2
cap = cv2.VideoCapture("/home/andi/FER/year5/PlayerTracking/sequences/demo-s.mp4")

def click_event(event, x, y, flags, params):
 
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', frame)


        
myFrameNumber = 10

# get total number of frames
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        
# check for valid frame number
if myFrameNumber >= 0 & myFrameNumber <= totalFrames:
    # set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES,myFrameNumber)

while True:
    ret, frame = cap.read()
    cv2.imshow("Video", frame)
    print(frame.shape)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break


        
cv2.destroyAllWindows()
